### `SimCSE`训练
- model: BERT/RoBERTa

In [1]:
from main.trainers.cse_trainer import Trainer
from transformers import BertTokenizer

/root/miniconda3/envs/stextsim/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = BertTokenizer.from_pretrained('/root/model/bert_unsup')
trainer = Trainer(tokenizer=tokenizer,
                  from_pretrained='/root/model/bert_unsup',
                  data_present_path='./dataset/present.json',
                  max_seq_len=32,
                  hard_negative_weight=0,
                  batch_size=64,
                  temp=0.05,
                  data_name='WikiSTS',
                  task_name='SimCSE_Wiki_unsup')

for i in trainer(num_epochs=15, lr=2e-5, gpu=[0], eval_call_step=lambda x: x % 250 == 0):
    a = i

Train: 15/15: 100%|██████████| 32/32 [00:17<00:00,  1.86it/s, train_loss=0.517]


如果要单独验证请注释掉上述训练的for循环，然后运行下面的代码

In [3]:
trainer.eval(0, is_eval=True)

Eval: 1: 100%|██████████| 32/32 [00:09<00:00,  3.41it/s, Spearman=0.81, eval_acc=0.962, eval_loss=3.3, f1=0.966, precision=0.937, recall=0.996]  

F1: 0.9656729850592274, Precision: 0.9368974583616398, Recall: 0.9962721341938838


([0.7994250655174255,
  0.8511407971382141,
  0.08075308054685593,
  0.9778224229812622,
  0.1632455438375473,
  0.8975945711135864,
  0.8739565014839172,
  0.9233299493789673,
  0.8038058280944824,
  0.02370545081794262,
  0.8023916482925415,
  0.1274893581867218,
  0.8585975766181946,
  0.9379019141197205,
  -0.11704538017511368,
  0.9269180297851562,
  0.07827591150999069,
  0.9780994653701782,
  0.9701988101005554,
  0.8946804404258728,
  0.8715289235115051,
  0.8402402997016907,
  0.9574214220046997,
  0.8795698285102844,
  0.9364215135574341,
  0.875645101070404,
  0.9274947047233582,
  0.32637205719947815,
  0.9609537124633789,
  0.8520881533622742,
  0.24394463002681732,
  0.29676228761672974,
  0.960117518901825,
  0.973615288734436,
  0.7478406429290771,
  0.9453725814819336,
  0.8995208740234375,
  0.8635640144348145,
  0.8092207908630371,
  0.9221453070640564,
  0.934836208820343,
  0.6660947203636169,
  0.9060733914375305,
  0.9720228314399719,
  0.9205541014671326,
  0.93

### 使用`SentEval`评估模型性能

支持通过三种方式执行评估：

- 1. 通过bash脚本执行, 其中`tokenizer_path`缺省时默认使用`model_name_or_path`指定的模型的tokenizer

In [ ]:
!python main/evaluation.py --model_name_or_path='./model/bert-base-uncased' --tokenizer_path='./model/bert-base-uncased'

- 2. 通过`ipynb`执行, 其中在默认情况下将会根据`model_name_or_path`自动选择模型类型

In [ ]:
from main.evaluation import *

model_path = './model/bert-base-uncased'
tokenizer_path = './model/bert-base-uncased'

main([
    '--model_name_or_path', model_path,
    '--tokenizer_path', tokenizer_path,
])

- 3. 也可以通过自定义`model`传入到`main()`函数来执行评估, 若采用本项目中定义的模型建议采用以下方式执行评估, 否则可能会出现丢失部分模型参数的情况

In [ ]:
from main.evaluation import *
from main.models.simcse import SimCSE

model_path = './save_model/SimCSE_Wiki_unsup/simcse_best'
tokenizer_path = './model/bert-base-uncased'
model = SimCSE(from_pretrained=model_path,
                                pooler_type='cls')

main([
    '--model_name_or_path', model_path,
    '--tokenizer_path', tokenizer_path,
], model)

### 查看保存模型的权重

In [ ]:
import torch

# 指定预训练模型的路径
model_path = 'save_model/SimCSE_Wiki_unsup/simcse_best/pytorch_model.bin'

# 使用 torch.load 加载模型
state_dict = torch.load(model_path, map_location='cpu')

# 打印模型的键（参数的名称）
print("Model Keys:", state_dict.keys())

# 打印模型的详细信息
for key, value in state_dict.items():
    print(f"Key: {key}, Shape: {value.shape}")

### `NLI`训练
- model: BERT/SBERT

In [ ]:
from main.trainers.sts_trainer import Trainer
from transformers import BertTokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('./model/chinese_wwm_ext')
trainer = Trainer(tokenizer=tokenizer,
                  from_pretrained='./model/chinese_wwm_ext',
                  model_type='bert',
                  data_present_path='./dataset/present.json',
                  max_seq_len=128,
                  hard_negative_weight=0,
                  batch_size=64,
                  temp=0.05,
                  data_name='CNSTS',
                  task_name='BERT_CNSTS')

for i in trainer(num_epochs=15, lr=5e-5, gpu=[0], eval_call_step=lambda x: x % 250 == 0):
    a = i

如果要单独验证请注释掉上述训练的for循环，然后运行下面的代码

In [ ]:
trainer.eval(0, is_eval=True)